## **Agrupamento de Séries Temporais de Evapotranspiração de Referência do Norte de Minas Gerais**

<center>
<table>

<img src="https://github.com/Hidrovales/Balanco_Hidrico/blob/main/Figuras/salinas_horizontal_jpg.jpg?raw=true" width="200"/>   <img src="https://github.com/Hidrovales/Balanco_Hidrico/blob/main/Figuras/COLORlogovertical.png?raw=true" width="200"/>
<tr>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</tr>
</table>

**Colaboradores:** Franciele A. Barbosa, Patrícia O. Lucas e Marcelo R. Vicente

**Notebook I =** Leitura e tratamento dos dados

As **séries temporais** são um tipo comum de dados dinâmicos que podem ser definidas como um conjunto de observações realizadas em sequência ao longo do tempo. A importância de seu **agrupamento** se destaca em:

*   Na descoberta de padrões, que podem disponibilizar informações valiosas através do conjunto de dados das séries temporais
*   Técnica exploratória e também como sub-rotina em algoritmos de mineração de dados mais complexos, como descoberta de regras, indexação, classificação e detecção de irregularidades

A **evapotranspiração** *($ET$)* é a combinação de dois processos, a evaporação e a transpiração. A **evapotranspiração de referência** ($ET_0$) é a taxa de evapotranspiração de uma superfície de referência. Onde a superfície de referência é semelhante a uma grande área de grama verde em crescimento, de altura uniforme, sombreando completamente o solo e com disponibilidade ilimitada de umidade do solo. A $ET_0$ é influenciada apenas por fatores climáticos, portanto, pode ser calculada a partir de dados meteorológicos.

## **Objetivos:**

Esse trabalho tem como objetivo a comparação dos métodos de agrupamento ***K-means*** e Hierárquico de ***Ward*** para agrupamento de séries temporais de evapotranspiração de referência.

## **Base de dados**

O conjunto de dados inclui dados diários observados coletados de medidores de chuva, bem como estações meteorológicas convencionais e automáticas do período de **1 de janeiro de 1980 a 31 de dezembro de 2016**. Os dados estão organizados em **5285 arquivos** de formato **.csv** e cada arquivo possui duas colunas: **time (string)** e **ETo (float)**.

A base de dados foi extraída do artigo: **Daily gridded meteorological variables in Brazil (1980–2013)**, publicado em 08 de outubro de 2015, pelo International Journal of Climatology. Onde pode ser analisado detalhamente como os foram dados obtidos.

In [1]:
import glob
import pandas as pd
from datetime import datetime
from scipy.stats import zscore
import numpy as np

## **Procura e Leitura de arquivos**

Foi utilizado o **módulo glob**, para encontrar a partir de um caminho todos os 5285 arquivos que possuem nome que correspondem a um padrão especificado (lat). 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Procura todos os arquivos que estão nomeados com 'lat' no caminho especificado
arquivos = glob.glob("/content/drive/My Drive/TCC/data_mg/lat*.csv")

In [4]:
#Criação da lista para o armazenamento da leitura de cada arquivo
arquivos_df = []

#Laço de repetição que pega cada arquivo (x) dos arquivos, faz a leitura e armazena na lista (arquivos_df)
for x in arquivos:
  temp_df = pd.read_csv(x)
  arquivos_df.append(temp_df)

In [ ]:
arquivos_df

## **Conversão de tipo**

Cada arquivo possui **duas colunas** importantes: 

1.   **time:** do tipo string *('xxxx-xx-xx')*, onde possui a data da ETo
2.   **ETo:** do tipo float, onde possui o valor da ETo

Para melhor manipulação dos dados, foi necessária a **conversão** do **tipo string** da coluna **time** para o **tipo datetime**, e para tal foi utilizado a biblioteca **datetime**.


In [5]:
#Criação da lista para armazenar os arquivos em que o tipo da coluna for convertido para datetime
arquivos_conv = []

#Laço de repetição que pega cada arquivo (a) da lista (arquvios_df), converte o tipo da coluna para datetime de cada um e armazena na lista (arquivos_conv)
for a in arquivos_df:
  a['time'] = pd.to_datetime(a['time'], format= '%Y-%m-%d')
  arquivos_conv.append(a)

In [ ]:
arquivos_conv

## **Exclusão de linhas**

Cada arquivo, possui dados que começam de **janeiro de 1980** e vão até a **metade de 2017**. Para melhor tratamento dos dados, foi necessária a **exclusão das linhas** que possuíam dados **do ano de 2017**. Resultando em arquvivos que vão de **janeiro de 1980** até **dezembro de 2016**.

In [6]:
#Criação da lista para armazenar os arquivos em que o ano de 2017 foi excluído
arquivos_sem2017 = []

#Laço de repetição que pega cada arquivo (e) da lista (arquivos_conv) e seleciona somente os dados que possuem a data !=2017 e armazena na lista (arquivos_sem2017)
for e in arquivos_conv:
  result = e[e['time'].dt.year != 2017]
  arquivos_sem2017.append(result)

In [ ]:
arquivos_sem2017

## **Cálculo da média**
O agrupamento de séries temporais pode apresentar problemas em relação a **alta dimensionalidade**. Dessa forma, reduzir a dimensionalidade é uma técnica importante quando se trabalha com métodos de agrupamento, pois reduz os requisitos de memória e acelera significativamente o cálculo da distância entre os dados brutos. 

Para tal, foi necessrário calcular a **média da ETo** de cada mês de cada arquivo.

*   **Exemplo:** calcular a média do mês de janeiro/fevereiro, etc.

Para tal, foi utilizada a função **groupby()** do pandas, onde é possível selecionar grupos iguais (mês de janeiro/fevereiro, etc) de uma coluna específica, que nosso caso é a coluna **'time'** e calcular a média da coluna desejada, que nosso caso é a coluna da **ETo**.

E como resultado, foi obtido **12 valores para cada 5285 arquivos**.

In [7]:
#Criação da lista para armazenar os arquivos em que a média de cada mês foi calculada
arquivos_media = []

#Laço de repetição que pega cada arquivo (b) da lista (arquivos_sem2017) e calcula a média da ETo para cada mês e armazena na lista (arquivos_media)
for b in arquivos_sem2017:
  result1 = b.groupby(b.time.dt.month)['ETo'].mean()
  arquivos_media.append(result1)

In [ ]:
arquivos_media

## **Normalização Z**
A **normalização** é utilizada para corrigir possíveis distorções que os dados brutos podem apresentar. Entre as técnicas para normalizar e modificar séries temporais, a fim de corrigir distorções está a: 

*  **Normalização Z:** remove imperfeições e diferenças de escala que surgem a partir da geração das séries temporais, sejam estas obtidas por meio de medições ou de algum tipo de transformação de dados.

Para normalizar os 12 dados de cada arquivo da base de dados, foi utilizada a função **zscore** da biblioteca **scipy.stats**.



In [ ]:
#Criação da lista para armazenar os arquivos em que a normalização foi aplicada
arquivos_zscore = []

#Laço de repetição que pega cada arquivo (n) da lista (arquivos_media) e normaliza (zscore) e armazena na lista (arquivos_zscore)
for n in arquivos_zscore:
  normaliza = zscore(n)
  arquivos_zscore.append(normaliza)

In [ ]:
arquivos_zscore